In [26]:
import pandas as pd
import re
import string

In [27]:
#2 adet csv dosyamız olduğu için yalan haberleri ayrı gerçek haberleri ayrı ayrı alıyoruz. verilerimiz arasında bulunmayan değer kolonu ile gerçek haberleri yalan haberleri 0 diye belritiyoruz.
yalan = pd.read_csv(r"C:\Users\Samet\Desktop\fake new\Fake.csv") 
yalan["deger"] = 0 #değer isimli bir kolon oluşturarak yalan haberlere 0 değeri veriyoruz.

In [28]:
gerçek = pd.read_csv(r"C:\Users\Samet\Desktop\fake new\True.csv") #gerçek haberleri okuyoruz.
gerçek["deger"] =1 #değer isimli bir kolonda gerçek haberlere 1 değeri veriyoruz.

In [29]:
data = pd.concat([yalan,gerçek],axis = 0) #ayrı değişkenlere atadığımız değişkenleri 0 ekseninde birleştiriyoruz.

In [30]:
data.isna().sum() #artık toplu olarak tek değişkende bulunan verilerin içinde nan değer var mı diye kontrol ediyoruz.

title      0
text       0
subject    0
date       0
deger      0
dtype: int64

In [31]:
#önce yalan haberler sonra yalan haberler verileri sıra ile geldiği için tüm verileri karıştırıyoruz.
data = data.sample(frac = 1) 

In [32]:
data = pd.DataFrame(data) #verilerimizi dataframe e çeviriyoruz.

In [33]:
data.shape # verinin büyüklüğünü ekrana basıyoruz.

(44898, 5)

In [16]:
#öncelikle text kolonundaki tüm değişkenleri küçük harfli yapıyoruz ardından bazı özel karakterleri boşluk olarak değiştiriyoruz.
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [17]:
data["text"] = data["text"].apply(wordopt) #text kolonundaki verileri bu işleme sokuyoruz.

In [18]:
x = data["text"] #bağımsız değişkenimizi x isimli değişkene atıyoruz.
y = data["deger"] #bağımlı değişkenimizi y isimli değişkene atıyoruz.

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25) #verilerimzi %25 oranında train ve test olarak ayırıyoruz. 

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
#text verilerimizi modeli eğitebilmek için vektör haline getiriyoruz.
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [22]:
from sklearn.linear_model import LogisticRegression
#modeli eğitiyoruz.
lr = LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [23]:
predict = lr.predict(xv_test) #tahminleri alıyoruz.

In [24]:
from sklearn.metrics import confusion_matrix 

In [25]:
confusion_matrix(y_test,predict) #modelin başarı oranını test ediyoruz

array([[5777,   79],
       [  71, 5298]], dtype=int64)